# Dynamic error correction
This sample demonstrates how to create a 3 qubit repetition code that can be used to detect and correct bit flip errors. It leverages integrated hybrid computing features to count the number of times error correction was performed while the state of a logical qubit register is coherent.

Find more about integrated quantum computing at https://aka.ms/AQ/Hybrid/Docs

In this program you will learn to:
1. Encode a logical qubit using a 3 qubit repetition code.
2. Perform error correction when a bit flip error occurs.
3. Use classical compute to count the number of times error correction was performed.
## Connect to the Azure Quantum workspace
First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in Azure Portal.

Next, you can run `%azure.connect` to connect to the workspace and display the list of provisioned targets that support execution of Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace.

In [ ]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

## Open the required Q# namespaces
First, open the Q# namespaces that provide the necessary operations to implement this sample.


In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

## Encode a logical qubit using a repetition code
The following operation is used to encode a 3 qubit register into a single logical qubit using an error correcting repetition code.

In [ ]:
operation Encode(register : Qubit[]) : Unit is Adj {
    CNOT(register[0], register[1]);
    CNOT(register[0], register[2]);
}

## Implement an operation that could introduce errors
In order to test our error correction code, we can apply an operation to our logical qubit that is equivalent to applying an identity operation.
To achieve this, we can apply a few rotations about the x-axis such that after applying them the logical qubit is in the same state it was intially in.
Since the `Rx` operation has a $4\pi$ period, applying a $\frac{\pi}{2}$ rotation 8 times should return the qubit to its original state.
Note that since the register represents a logical qubit, rotations are applied to each qubit in the reguster.

The errors introduced by applying these rotations is what our program will attempt to correct.

In [ ]:
operation ApplyRotationalIdentity(register : Qubit[]) : Unit is Adj
{
    let theta = PI() * 0.5;
    for i in 1 .. 8 {
        for qubit in register
        {
            Rx(theta, qubit);
        }
    }
}


## Measure a bit flip error syndromeThe following operation measures a bit flip error syndrome by checking the parities between qubits 0 and 1, and between qubits 1 and 2.

This operation uses a 2 qubit auxiliary register and leverages integrated hybrid computing capabilities: mid-program qubit measurement and qubit re-use. For an error correction program, these features reduce the number of qubits needed because otherwise we would have to use 2 additional qubits every time we wanted to measure the error syndrome.

In [ ]:
operation MeasureSyndrome(logicalRegister : Qubit[], auxiliaryRegister : Qubit[]) : (Result, Result)
{
    ResetAll(auxiliaryRegister);
    CNOT(logicalRegister[0], auxiliaryRegister[0]);
    CNOT(logicalRegister[1], auxiliaryRegister[0]);
    CNOT(logicalRegister[1], auxiliaryRegister[1]);
    CNOT(logicalRegister[2], auxiliaryRegister[1]);
    let parity01 = MResetZ(auxiliaryRegister[0]);
    let parity12 = MResetZ(auxiliaryRegister[1]);
    return (parity01, parity12);
}


## Use error syndrome to correct errors
The following operation applies a corrective operation to a specific qubit in the register depending on the provided error syndrome and returns a boolean that represents whether any a correction was performed on any of the qubits
.
This operation uses another integrated hybrid computing capability, branching based on qubit measurements. As we can observe, a different quantum operation is performed depending on the measurement results.

In [ ]:
operation CorrectError(register : Qubit[], parity01 : Result, parity12 : Result) : Bool {
    if (parity01 == One and parity12 == Zero) {
         X(register[0]); 
    }
    elif (parity01 == One and parity12 == One) {
        X(register[1]);
    }
    elif (parity01 == Zero and parity12 == One) {
        X(register[2]);
    }

    return parity01 == One or parity12 == One;
}

## 6. Define the DynamicBitFlipCode operation
Take all of the operations defined above and iterate multiple times:
1. Apply a logical operation that may introduce errors.
2. Attempt to correct introduced error and increment error count accordingly.
3. Measure the register to read the logical qubit value.
4. Report the value of each logical qubit.

In [ ]:
operation DynamicBitFlipCode() : (Result, Int){
    // Create a registers that represents a logical qubit.
    use localRegister = Qubit[3];

    // Apply several unitary operations to the encoded qubits performing error correction between each application.
    mutable corrections = 0;
    within {
        // Encode/Decode logical qubit.
        Encode(localRegister);
    }
    apply {
        LogicalX(localRegister);
        let iterations = 5;
        for _ in 1 .. iterations {
            // Apply unitary operations.
            ApplyLogicalOperation(localRegister);

            // Perform error correction and increase the counter if a correction was made.
            let (parityA01, parityA12) = MeasureSyndrome(localRegister);
            let correctedError = CorrectError(localRegister, parityA01, parityA12);
            if (correctedError) {
                set corrections += 1;
            }
        }
    }

    // Measure the first qubit in register and return value.
    let result = MResetZ(localRegister[0]);
    ResetAll(localRegister);
    return (result, corrections);
}

## 7. Submit the DynamicBitFlipCode operation
Using 50 shots, as configured below, **submitting this program will use approximately 11.31 units (HQCs or eHQCs)**.<br/>
If the job has not been completed after the client times out (300 seconds), you can query the status using `%azure.status` followed by querying the output using `%azure.output`.<br/>
If the kernel was restarted after the program had been submitted, you will have to use `%azure.status <job_id>` to get the status of the job and `%azure.output <job_id>` to retrieve the results.<br/>
You can also check the status of the job under the **Job management** section of the portal.

**Understanding the results**<br/>
The histogram reports the ratio of the various outputs.</br>
For instance, if you get:</br>
- `Result: (1,0) Frequency: 0.92` means that in 92% of the total number of shots the operation returned a qubit with the value One and that no error correction was necessary.</br>
- `Result: (0,2) Frequency: 0.02` means that in 2% of the total number of shots the operation returned a qubit with the value Zero and that 2 error corrections were performed. The error correction failed as the value flipped.</br>
- `Result: (0,0) Frequency: 0` means that in 2% of the total number of shots the operation returned a qubit with the value One and that 2 error corrections were performed. The error correction failed as the value flipped.</br>
- `Result: (1,2) Frequency: 0.02` means that in 2% of the total number of shots the operation returned a qubit with the value One and that 2 error corrections were performed.

In [ ]:
%azure.target quantinuum.sim.h1-1e
%azure.target-capability AdaptiveExecution
%azure.execute DynamicBitFlipCode shots=50 timeout=300 poll=10

In [ ]:
%azure.status

In [ ]:
%azure.output

# ↗ Run an advanced experiment - Iterative phase estimation
Now that you have learned the basics of Integrated hybrid, experiment with the Iterative phase estimation sample available in the sample gallery as well.</br>
To learn more, please refer to the [online documentation](https://aka.ms/AQ/Hybrid/Docs).